In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from numpy import *

# Imports
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from gensim.models import word2vec

[nltk_data] Downloading package stopwords to /home/aman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
y_train = train.age.values
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['age'], axis=1, inplace=True)

In [4]:
data['year'] = data['date'].str[-4:]
data.head()

,date,gender,post.id,sign,text,topic,user.id,year
0,"14,May,2004",male,1,Leo,"Info has been found (+/- 100 pages,...",Student,11869,2004
1,"13,May,2004",male,2,Leo,These are the team members: Drewe...,Student,11869,2004
2,"12,May,2004",male,3,Leo,In het kader van kernfusie op aarde...,Student,11869,2004
3,"12,May,2004",male,4,Leo,testing!!! testing!!!,Student,11869,2004
4,"11,June,2004",male,5,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,InvestmentBanking,16332,2004


In [5]:
def sentence_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return words

In [6]:
# Word2Vec expects single sentences, 
# each one as a list of words. 
# In other words, the input format is a list of lists.
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(sentence_to_wordlist(raw_sentence, remove_stopwords))
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [7]:
sentences = []  # Initialize an empty list of sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print ("Parsing sentences from data set")
for review in data["text"]:
    sentences += review_to_sentences(review, tokenizer, remove_stopwords=True)

Parsing sentences from data set


In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_40minwords_10context"
model.save(model_name)

2018-09-30 04:01:28,882 : INFO : collecting all words and their counts
2018-09-30 04:01:28,883 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-30 04:01:28,898 : INFO : PROGRESS: at sentence #10000, processed 82036 words, keeping 13964 word types
2018-09-30 04:01:28,923 : INFO : PROGRESS: at sentence #20000, processed 159029 words, keeping 20864 word types
2018-09-30 04:01:28,941 : INFO : PROGRESS: at sentence #30000, processed 226837 words, keeping 24584 word types
2018-09-30 04:01:28,976 : INFO : PROGRESS: at sentence #40000, processed 313265 words, keeping 29625 word types
2018-09-30 04:01:29,015 : INFO : PROGRESS: at sentence #50000, processed 395778 words, keeping 33557 word types
2018-09-30 04:01:29,057 : INFO : PROGRESS: at sentence #60000, processed 471737 words, keeping 37455 word types


Training model...


2018-09-30 04:01:29,100 : INFO : PROGRESS: at sentence #70000, processed 560271 words, keeping 41216 word types
2018-09-30 04:01:29,133 : INFO : PROGRESS: at sentence #80000, processed 616687 words, keeping 42590 word types
2018-09-30 04:01:29,162 : INFO : PROGRESS: at sentence #90000, processed 678264 words, keeping 43944 word types
2018-09-30 04:01:29,192 : INFO : PROGRESS: at sentence #100000, processed 760981 words, keeping 46929 word types
2018-09-30 04:01:29,244 : INFO : PROGRESS: at sentence #110000, processed 835924 words, keeping 50060 word types
2018-09-30 04:01:29,284 : INFO : PROGRESS: at sentence #120000, processed 917511 words, keeping 52107 word types
2018-09-30 04:01:29,318 : INFO : PROGRESS: at sentence #130000, processed 1004723 words, keeping 55168 word types
2018-09-30 04:01:29,352 : INFO : PROGRESS: at sentence #140000, processed 1084802 words, keeping 58078 word types
2018-09-30 04:01:29,384 : INFO : PROGRESS: at sentence #150000, processed 1157493 words, keeping 

2018-09-30 04:01:32,347 : INFO : PROGRESS: at sentence #790000, processed 5917941 words, keeping 143177 word types
2018-09-30 04:01:32,403 : INFO : PROGRESS: at sentence #800000, processed 5993607 words, keeping 144176 word types
2018-09-30 04:01:32,468 : INFO : PROGRESS: at sentence #810000, processed 6077877 words, keeping 145384 word types
2018-09-30 04:01:32,525 : INFO : PROGRESS: at sentence #820000, processed 6147102 words, keeping 145948 word types
2018-09-30 04:01:32,586 : INFO : PROGRESS: at sentence #830000, processed 6252054 words, keeping 149985 word types
2018-09-30 04:01:32,624 : INFO : PROGRESS: at sentence #840000, processed 6325323 words, keeping 150886 word types
2018-09-30 04:01:32,653 : INFO : PROGRESS: at sentence #850000, processed 6403982 words, keeping 151631 word types
2018-09-30 04:01:32,691 : INFO : PROGRESS: at sentence #860000, processed 6472429 words, keeping 152516 word types
2018-09-30 04:01:32,731 : INFO : PROGRESS: at sentence #870000, processed 655904

2018-09-30 04:01:34,761 : INFO : PROGRESS: at sentence #1500000, processed 11412658 words, keeping 211554 word types
2018-09-30 04:01:34,787 : INFO : PROGRESS: at sentence #1510000, processed 11503710 words, keeping 212421 word types
2018-09-30 04:01:34,817 : INFO : PROGRESS: at sentence #1520000, processed 11587029 words, keeping 213267 word types
2018-09-30 04:01:34,849 : INFO : PROGRESS: at sentence #1530000, processed 11676759 words, keeping 214884 word types
2018-09-30 04:01:34,869 : INFO : PROGRESS: at sentence #1540000, processed 11745060 words, keeping 215342 word types
2018-09-30 04:01:34,896 : INFO : PROGRESS: at sentence #1550000, processed 11822892 words, keeping 216358 word types
2018-09-30 04:01:34,916 : INFO : PROGRESS: at sentence #1560000, processed 11889382 words, keeping 217005 word types
2018-09-30 04:01:34,933 : INFO : PROGRESS: at sentence #1570000, processed 11943009 words, keeping 217115 word types
2018-09-30 04:01:34,956 : INFO : PROGRESS: at sentence #1580000,

...

2018-09-30 04:02:13,558 : INFO : EPOCH 1 - PROGRESS: at 24.54% examples, 1121787 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:02:14,564 : INFO : EPOCH 1 - PROGRESS: at 26.19% examples, 1117855 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:02:15,577 : INFO : EPOCH 1 - PROGRESS: at 27.92% examples, 1113363 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:02:16,597 : INFO : EPOCH 1 - PROGRESS: at 29.78% examples, 1112060 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:02:17,600 : INFO : EPOCH 1 - PROGRESS: at 31.45% examples, 1109666 words/s, in_qsize 16, out_qsize 1
2018-09-30 04:02:18,618 : INFO : EPOCH 1 - PROGRESS: at 33.18% examples, 1107567 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:02:19,621 : INFO : EPOCH 1 - PROGRESS: at 34.95% examples, 1106439 words/s, in_qsize 16, out_qsize 1
2018-09-30 04:02:20,623 : INFO : EPOCH 1 - PROGRESS: at 36.53% examples, 1105178 words/s, in_qsize 14, out_qsize 1
2018-09-30 04:02:21,625 : INFO : EPOCH 1 - PROGRESS: at 38.17% examples, 1103693

2018-09-30 04:03:18,771 : INFO : EPOCH 2 - PROGRESS: at 32.96% examples, 1039720 words/s, in_qsize 12, out_qsize 3
2018-09-30 04:03:19,789 : INFO : EPOCH 2 - PROGRESS: at 34.71% examples, 1040608 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:20,796 : INFO : EPOCH 2 - PROGRESS: at 36.18% examples, 1038342 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:21,817 : INFO : EPOCH 2 - PROGRESS: at 37.44% examples, 1029272 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:22,820 : INFO : EPOCH 2 - PROGRESS: at 39.19% examples, 1031445 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:23,827 : INFO : EPOCH 2 - PROGRESS: at 40.94% examples, 1035283 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:24,848 : INFO : EPOCH 2 - PROGRESS: at 42.80% examples, 1042425 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:25,854 : INFO : EPOCH 2 - PROGRESS: at 44.76% examples, 1051454 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:03:26,859 : INFO : EPOCH 2 - PROGRESS: at 46.36% examples, 1049684

2018-09-30 04:04:24,219 : INFO : EPOCH 3 - PROGRESS: at 56.17% examples, 1239203 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:04:25,221 : INFO : EPOCH 3 - PROGRESS: at 58.02% examples, 1238977 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:04:26,233 : INFO : EPOCH 3 - PROGRESS: at 60.20% examples, 1238830 words/s, in_qsize 16, out_qsize 2
2018-09-30 04:04:27,257 : INFO : EPOCH 3 - PROGRESS: at 62.30% examples, 1239782 words/s, in_qsize 13, out_qsize 2
2018-09-30 04:04:28,258 : INFO : EPOCH 3 - PROGRESS: at 64.27% examples, 1240056 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:04:29,266 : INFO : EPOCH 3 - PROGRESS: at 66.13% examples, 1238903 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:04:30,271 : INFO : EPOCH 3 - PROGRESS: at 67.79% examples, 1233442 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:04:31,271 : INFO : EPOCH 3 - PROGRESS: at 69.44% examples, 1228469 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:04:32,283 : INFO : EPOCH 3 - PROGRESS: at 71.22% examples, 1224759

2018-09-30 04:05:28,921 : INFO : EPOCH 4 - PROGRESS: at 64.12% examples, 1044784 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:05:29,933 : INFO : EPOCH 4 - PROGRESS: at 65.84% examples, 1046236 words/s, in_qsize 14, out_qsize 1
2018-09-30 04:05:30,942 : INFO : EPOCH 4 - PROGRESS: at 67.53% examples, 1046584 words/s, in_qsize 16, out_qsize 2
2018-09-30 04:05:31,942 : INFO : EPOCH 4 - PROGRESS: at 69.23% examples, 1047419 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:05:32,943 : INFO : EPOCH 4 - PROGRESS: at 70.99% examples, 1048718 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:05:33,947 : INFO : EPOCH 4 - PROGRESS: at 72.72% examples, 1050336 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:05:34,953 : INFO : EPOCH 4 - PROGRESS: at 74.33% examples, 1050990 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:05:35,973 : INFO : EPOCH 4 - PROGRESS: at 75.96% examples, 1051491 words/s, in_qsize 16, out_qsize 1
2018-09-30 04:05:36,974 : INFO : EPOCH 4 - PROGRESS: at 77.75% examples, 1052275

2018-09-30 04:06:33,865 : INFO : EPOCH 5 - PROGRESS: at 69.02% examples, 1022196 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:06:34,872 : INFO : EPOCH 5 - PROGRESS: at 70.65% examples, 1022212 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:06:35,874 : INFO : EPOCH 5 - PROGRESS: at 72.28% examples, 1022378 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:06:36,891 : INFO : EPOCH 5 - PROGRESS: at 73.83% examples, 1022158 words/s, in_qsize 14, out_qsize 1
2018-09-30 04:06:37,902 : INFO : EPOCH 5 - PROGRESS: at 75.41% examples, 1023450 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:06:38,929 : INFO : EPOCH 5 - PROGRESS: at 77.17% examples, 1024242 words/s, in_qsize 16, out_qsize 0
2018-09-30 04:06:39,944 : INFO : EPOCH 5 - PROGRESS: at 78.93% examples, 1025180 words/s, in_qsize 15, out_qsize 0
2018-09-30 04:06:40,952 : INFO : EPOCH 5 - PROGRESS: at 80.63% examples, 1026499 words/s, in_qsize 12, out_qsize 3
2018-09-30 04:06:41,956 : INFO : EPOCH 5 - PROGRESS: at 82.04% examples, 1025117

In [9]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = int(0)
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [10]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [ ]:
model = word2vec.Word2Vec.load("100features_40minwords_10context").wv
clean_train_reviews = []
for review in train["text"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["text"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

2018-09-30 04:07:37,091 : INFO : loading Word2Vec object from 100features_40minwords_10context
2018-09-30 04:07:37,430 : INFO : loading wv recursively from 100features_40minwords_10context.wv.* with mmap=None
2018-09-30 04:07:37,431 : INFO : setting ignored attribute vectors_norm to None
2018-09-30 04:07:37,431 : INFO : loading vocabulary recursively from 100features_40minwords_10context.vocabulary.* with mmap=None
2018-09-30 04:07:37,432 : INFO : loading trainables recursively from 100features_40minwords_10context.trainables.* with mmap=None
2018-09-30 04:07:37,432 : INFO : setting ignored attribute cum_table to None
2018-09-30 04:07:37,433 : INFO : loaded 100features_40minwords_10context


Review 0 of 442961


/home/aman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 442961
Review 2000 of 442961
Review 3000 of 442961
Review 4000 of 442961
Review 5000 of 442961
Review 6000 of 442961
Review 7000 of 442961
Review 8000 of 442961
Review 9000 of 442961
Review 10000 of 442961
Review 11000 of 442961
Review 12000 of 442961
Review 13000 of 442961
Review 14000 of 442961
Review 15000 of 442961
Review 16000 of 442961
Review 17000 of 442961
Review 18000 of 442961
Review 19000 of 442961
Review 20000 of 442961
Review 21000 of 442961
Review 22000 of 442961
Review 23000 of 442961
Review 24000 of 442961
Review 25000 of 442961
Review 26000 of 442961
Review 27000 of 442961
Review 28000 of 442961
Review 29000 of 442961
Review 30000 of 442961
Review 31000 of 442961
Review 32000 of 442961
Review 33000 of 442961
Review 34000 of 442961
Review 35000 of 442961
Review 36000 of 442961
Review 37000 of 442961
Review 38000 of 442961
Review 39000 of 442961
Review 40000 of 442961
Review 41000 of 442961
Review 42000 of 442961
Review 43000 of 442961
Review 44000 of 4429

Review 347000 of 442961
Review 348000 of 442961
Review 349000 of 442961
Review 350000 of 442961
Review 351000 of 442961
Review 352000 of 442961
Review 353000 of 442961
Review 354000 of 442961
Review 355000 of 442961
Review 356000 of 442961
Review 357000 of 442961
Review 358000 of 442961
Review 359000 of 442961
Review 360000 of 442961
Review 361000 of 442961
Review 362000 of 442961
Review 363000 of 442961
Review 364000 of 442961
Review 365000 of 442961
Review 366000 of 442961
Review 367000 of 442961
Review 368000 of 442961
Review 369000 of 442961
Review 370000 of 442961
Review 371000 of 442961
Review 372000 of 442961
Review 373000 of 442961
Review 374000 of 442961
Review 375000 of 442961
Review 376000 of 442961
Review 377000 of 442961
Review 378000 of 442961
Review 379000 of 442961
Review 380000 of 442961
Review 381000 of 442961
Review 382000 of 442961
Review 383000 of 442961
Review 384000 of 442961
Review 385000 of 442961
Review 386000 of 442961
Review 387000 of 442961
Review 388000 of

In [33]:
# Initializing and fitting logistic regression model
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [34]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [39]:
from numpy import *
where_are_NaNs = isnan(trainDataVecs)
trainDataVecs[where_are_NaNs] = 0

In [43]:
np.savetxt('test1.txt', a, fmt='%d')

1000

In [3]:
b = np.loadtxt('trainDataVecs.txt', dtype=int)

In [6]:
len(train['text'])

NameError: name 'train' is not defined

In [ ]:
preds = lr.predict(test_data_features)

In [ ]:
solution = pd.DataFrame({"user.id":test["user.id"], "age":preds})

In [ ]:
solution

In [ ]:
solution = solution.groupby('user.id', as_index=False)['age'].mean()

In [ ]:
solution.to_csv("solutions/baselinelr.csv", index = False)